# Challenge Creation

### Import necessary packages

In [1]:
import torch
from torch import nn
import torch.optim as optim
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

### ENIGMABOT Class

In [3]:
class ENIGMABOT(nn.Module):
    def __init__(self):
        super(ENIGMABOT, self).__init__()

        self.embedding = nn.Embedding(256, 32)
        self.lstm = nn.LSTM(32,128, 2, batch_first=True, dropout=.1)
        self.fc = nn.Sequential(
            nn.Linear(128, 128),
            nn.LayerNorm(128),
            nn.ELU(),
            nn.Linear(128, 256)
        )

    def forward(self, input):
        embedded = self.embedding(input)
        output = self.lstm(embedded)[0][:,-1,:]
        prediction = self.fc(output)
        return prediction

In [4]:
model = ENIGMABOT()
model.load_state_dict(torch.load("enigmabot_model.pt"))

<All keys matched successfully>

### Featurize function

In [5]:
def featurize(text):
    return torch.Tensor([[int(ord(x)) for x in text]]).long()

### Create the dataset

In [6]:
transmission = "9Zx4Qp 8Yw3Po 7Xv2On 6Wu1Nm 5Vt0Ml 4Us9Lk 3Tr8Kj 2Sq7Ji"
flag = ": ingeneer{IMMUNEBOOSTERSREADY.PATHOGENMUTATIONSTABILIZED.DISTRIBUTIONPLAN}"

text = transmission + flag

In [37]:
data = pd.DataFrame(columns=['sequence', 'target'])

for i in range(0, len(flag)):
    data.loc[i] = [text[:i + len(transmission)], text[i + len(transmission)]]

In [38]:
data

,sequence,target
0,9Zx4Qp 8Yw3Po 7Xv2On 6Wu1Nm 5Vt0Ml 4Us9Lk 3Tr8...,:
1,9Zx4Qp 8Yw3Po 7Xv2On 6Wu1Nm 5Vt0Ml 4Us9Lk 3Tr8...,
2,9Zx4Qp 8Yw3Po 7Xv2On 6Wu1Nm 5Vt0Ml 4Us9Lk 3Tr8...,i
3,9Zx4Qp 8Yw3Po 7Xv2On 6Wu1Nm 5Vt0Ml 4Us9Lk 3Tr8...,n
4,9Zx4Qp 8Yw3Po 7Xv2On 6Wu1Nm 5Vt0Ml 4Us9Lk 3Tr8...,g
...,...,...
70,9Zx4Qp 8Yw3Po 7Xv2On 6Wu1Nm 5Vt0Ml 4Us9Lk 3Tr8...,P
71,9Zx4Qp 8Yw3Po 7Xv2On 6Wu1Nm 5Vt0Ml 4Us9Lk 3Tr8...,L
72,9Zx4Qp 8Yw3Po 7Xv2On 6Wu1Nm 5Vt0Ml 4Us9Lk 3Tr8...,A
73,9Zx4Qp 8Yw3Po 7Xv2On 6Wu1Nm 5Vt0Ml 4Us9Lk 3Tr8...,N


### Training the model

In [39]:
# Instantiate the model
model = ENIGMABOT()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs
num_epochs = 100

# Training loop
for epoch in range(num_epochs):
    total_loss = 0
    for i in range(len(data)):
        sequence = data.loc[i, 'sequence']
        target = data.loc[i, 'target']

        # Featurize the input sequence
        input_tensor = featurize(sequence)

        # Forward pass
        output = model(input_tensor)
        loss = criterion(output, torch.LongTensor([ord(target)]))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Print the average loss for the epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(data)}')

Epoch [1/100], Loss: 4.6901111284891766
Epoch [2/100], Loss: 3.2945802942911784
Epoch [3/100], Loss: 2.8725102583567304
Epoch [4/100], Loss: 2.6518141214052835
Epoch [5/100], Loss: 2.2307767923672994
Epoch [6/100], Loss: 2.0916917792956036
Epoch [7/100], Loss: 2.057218872706095
Epoch [8/100], Loss: 1.4733907500902812
Epoch [9/100], Loss: 1.1863986877600352
Epoch [10/100], Loss: 0.860781389772892
Epoch [11/100], Loss: 0.6491092353065808
Epoch [12/100], Loss: 0.5153728369871775
Epoch [13/100], Loss: 0.42967908799648286
Epoch [14/100], Loss: 0.2983924443771442
Epoch [15/100], Loss: 0.27641010517875353
Epoch [16/100], Loss: 0.24774437223871548
Epoch [17/100], Loss: 0.21848746811350186
Epoch [18/100], Loss: 0.15501481983810664
Epoch [19/100], Loss: 0.08811107128858567
Epoch [20/100], Loss: 0.07343442837397257
Epoch [21/100], Loss: 0.052741703366239866
Epoch [22/100], Loss: 0.058771563284099104
Epoch [23/100], Loss: 0.10963926969096065
Epoch [24/100], Loss: 0.05716527589907249
Epoch [25/100]

### Save the model

In [40]:
torch.save(model.state_dict(), 'enigmabot_model.pt')